In [55]:
import plotly.graph_objects as go 
import numpy as np

In [102]:

class Maillage : 
    def __init__(self,X : np.ndarray ,Y : np.ndarray,Z : np.ndarray) :         
        if (size := X.size) == Y.size == Z.size : 
            X,Y,Z = X.flatten(),Y.flatten(),Z.flatten()
            self.X = X
            self.Y = Y
            self.Z = Z
            self.XYZ = X,Y,Z
            self.nb_points = size
            self.nb_points_droite = np.cbrt(size)
            self.nb_points_plan = self.nb_points_droite**2
            self.val_min = self[0]
            self.val_max = self[size-1]
            self.pas = round((self.val_max[0] - self.val_min[0]) / self.nb_points_droite, 2)
                 
    def __add__(self,maillage : "Maillage") -> "Maillage" : 
        X,Y,Z = np.concatenate((self.X,maillage.X)),np.concatenate((self.Y,maillage.Y)),np.concatenate((self.Z,maillage.Z))
        return Maillage(X,Y,Z)
    
    def __getitem__(self,index : int) -> "Maillage" : 
        return self.X[index],self.Y[index],self.Z[index]
    
    def __delitem__(self,index : int) : 
        X = np.concatenate((self.X[:index], self.X[index+1:]))
        Y = np.concatenate((self.Y[:index], self.Y[index+1:]))
        Z = np.concatenate((self.Z[:index], self.Z[index+1:]))
        
        maillage = Maillage(X,Y,Z)
        
        for attr in list(self.__dict__.keys()) : 
            setattr(self,attr, getattr(maillage,attr))
     
    def __eq__(self, maillage : "Maillage"):
        return np.array_equal(self.X, maillage.X) and \
               np.array_equal(self.Y, maillage.Y) and \
               np.array_equal(self.Z, maillage.Z)
               
    def add_point(self,point : np.ndarray) -> "Maillage" : 
        if point.shape == (3,) : 
            X = self.X + point[0]
            Y = self.Y + point[1]
            Z = self.Z + point[2]
            return Maillage(X,Y,Z) 
        raise ValueError("Un point est nécéssairement de dimension (3,)")
    
    def norm(self) -> np.ndarray : 
        return np.sqrt(self.X**2 + self.Y**2 + self.Z**2)
    
    def afficher(self):
        fig = go.Figure(data=[go.Scatter3d(x=self.X, y=self.Y, z=self.Z, mode='markers',marker=dict(size=2))])
        fig.show()

In [104]:

def creer_maillage_cartesien(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> Maillage:
    """
    Entrées:

        - x_min : la valeur minimale en x du maillage (Exemple : x_min = -2.5). Le signe est quelconque et la valeur doit être différente de x_max.
        - y_min : la valeur minimale en y du maillage (Exemple : y_min = -2.5). Le signe est quelconque et la valeur doit être différente de y_max.
        - z_min : la valeur minimale en z du maillage (Exemple : z_min = -2.5). Le signe est quelconque et la valeur doit être différente de z_max.

        - x_max : la valeur maximale en x du maillage (Exemple : x_max = 2.5). Le signe est quelconque et la valeur doit être différente de x_min.
        - y_max : la valeur maximale en y du maillage (Exemple : y_max = 2.5). Le signe est quelconque et la valeur doit être différente de y_min.
        - z_max : la valeur maximale en z du maillage (Exemple : z_max = 2.5). Le signe est quelconque et la valeur doit être différente de z_min.

        - pas_du_maillage : 
            
            * Distance séparant 2 points (non diagonnaux) du maillage. 
            
            * Il est de préférence un diviseur de (x_max - x_min) et de (y_max - y_min) et (z_max - z_min). 
            
            * Éviter un pas égal à 0.3 ou quelconque multiple de 0.3 ainsi que les nombres premiers par exemple. 
            
            * Sa valeur ne doit pas être plus élevée que la moitié de la plus petite distance coordonnée_max - coordonnée_min des 3 axes.
        
    Sorties:
    
        - [X,Y,Z] : C'est une liste qui contient 3 listes de NP.ARRAYS 2D. 
            
            * len(X) = len(Y) = len(Z) = nb_points_entre_y_min_y_max.
            
            * X[0] ou Y[0] ou Z[0] représente le plan (xOz) pour y = y_min, X[1] ou Y[1] ou Z[1] pour y = y_min + pas_du_maillage et ainsi 
                de suite jusqu'à y = y_max. 
            
            * De plus, pour chaque NP.ARRAY de X, Y ou Z, X[y_fixé][:,0], Y[y_fixé][:,0], Z[y_fixé][:,0] représente les droites parallèles 
                à l'axe (Ox) pour y = y_fixé et z = z_min. De même, X[y_fixé][:,1], Y[y_fixé][:,1], Z[y_fixé][:,1] représente les droites 
                parallèles à l'axe (Ox) pour y = y_fixé et z = z_min + pas_du_maillage. 
            
            * On a également X[y_fixé][0,:], Y[y_fixé][0,:], Z[y_fixé][0,:] représente les droites parallèles à l'axe (Oz) pour y = y_fixé et 
                x = x_min. De même, X[y_fixé][1,:], Y[y_fixé][1,:], Z[y_fixé][1,:] représente les droites parallèles à l'axe (Oz) pour y = y_fixé 
                et x = x_min + pas_du_maillage.
            
    Explication:
    
        La fonction creer_maillage_cartesien() crée une maillage de l'espace pour des limites et un pas donnés.
    """
    
    #Vérification de la cohérence des données.
    for min, max in zip([x_min,y_min,z_min],[x_max,y_max,z_max]):
        est_un_plan = False
        est_une_droite = False
        
        if min == max: #Sinon le maillage est soit: un plan, une droite ou un point ce qui ne nous intéresse pas ici.
            if est_un_plan:
                if est_une_droite:
                    raise ValueError("Le maillage est un point.")
                est_une_droite = True
            est_un_plan = True    
    
        if min > max:
            min, max = max, min #Nécessaire pour avoir des np.linspace(min, max, nb_points) cohérents.
        
        if not est_un_plan:   
            if pas_du_maillage > abs(max-min)/2:
                raise ValueError("pas_du_maillage est trop grand.") #Le cas écheant, le maillage ne serait pas bien défini.

    #Création du maillage.
    longueur_maillage_x = abs(x_max - x_min)
    longueur_maillage_y = abs(y_max - y_min)
    longueur_maillage_z = abs(z_max - z_min)
    
    try:
        nb_points_entre_x_min_x_max = int((longueur_maillage_x) / (pas_du_maillage)) + 1
    except:
        raise ZeroDivisionError
    try:
        nb_points_entre_y_min_y_max = int((longueur_maillage_y) / (pas_du_maillage)) + 1
    except:
        raise ZeroDivisionError
    try:
        nb_points_entre_z_min_z_max = int((longueur_maillage_z) / (pas_du_maillage)) + 1
    except:
        raise ZeroDivisionError
    
    axe_x = np.linspace(x_min, x_max, nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min, y_max, nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min, z_max, nb_points_entre_z_min_z_max)
    
    X,Y,Z = np.meshgrid(axe_x,axe_y,axe_z)
    
    #Permet de gérer les cas où pas_du_maillage engendre des points dont les coordonnées ont des décimales infinies comme 0.3.
    X = np.round(X, 1) 
    Y = np.round(Y, 1)
    Z = np.round(Z, 1)

    return Maillage(X,Y,Z)

cote = 2 #N'est pas une variable globale.
pas_du_maillage = 0.1 #N'est pas une variable globale.
test = creer_maillage_cartesien(-cote,cote,-cote,cote,-cote,cote,pas_du_maillage)  

In [99]:
def isoler_plan_cartesien(maillage : Maillage, plan : str, valeur_coupe : float) -> tuple:
    
    val_min_x, val_min_y, val_min_z = maillage.val_min
    val_max_x, val_max_y, val_max_z = maillage.val_max
    nb_plan = int(maillage.nb_points/maillage.nb_points_plan)
    
    if plan.strip().lower() in ["xoz","zox"]:
        if valeur_coupe == None:
            raise ValueError('Précisez la valeur de la côte du plan.')
        if valeur_coupe > val_max_y:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe < val_min_y:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe not in maillage.Y:
            raise ValueError("Le plan n'existe pas")
        val_indice_y = int(abs(val_min_y - valeur_coupe)/(maillage.pas) * maillage.nb_points_plan)
        
        return maillage[val_indice_y:val_indice_y+int(maillage.nb_points_plan)]
    
    if plan.strip().lower() in ["xoy","yox"]:
        if valeur_coupe == None:
            raise ValueError('Précisez la valeur de la côte du plan.')
        if valeur_coupe > val_max_z:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe < val_min_z:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe not in maillage.Z:
            raise ValueError("Le plan n'existe pas")
        val_indice_z = int(abs(val_min_z - valeur_coupe)/maillage.pas)
        
        liste = []
        for valeur in maillage.Y[::int(maillage.nb_points_plan)]:
            liste.append(float(valeur))
        
        return maillage.X[:int(maillage.nb_points_plan)], liste*nb_plan, maillage.Z[val_indice_z::int(maillage.nb_points_droite)]
    
    if plan.strip().lower() in ["yoz","zoy"]:
        if valeur_coupe == None:
            raise ValueError('Précisez la valeur de la côte du plan.')
        if valeur_coupe > val_max_x:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe < val_min_x:
            raise ValueError("Le plan n'existe pas.")
        if valeur_coupe not in maillage.X:
            raise ValueError("Le plan n'existe pas")
        
        liste_x = []
        for _ in range(int(maillage.nb_points_plan)):
            liste_x.append(float(valeur_coupe))
            
        liste_y = []
        for valeur in maillage.Y[::int(maillage.nb_points_plan)]:
            liste_temporaire = []
            liste_temporaire.append(float(valeur))
            liste_y += liste_temporaire*nb_plan
    
        return liste_x, liste_y, maillage.Z[:int(maillage.nb_points_plan)]
    
    raise ValueError("La string du plan ne convient pas. Vérifiez l'orthographe ou si le plan existe.")

In [101]:
X,Y,Z = isoler_plan_cartesien(test,"xoz",2)
fig = go.Figure(data=[go.Scatter3d(x=X, y=Y, z=Z, mode='markers',marker=dict(size=2))])
fig.show()